In [1]:
#keras
import tensorflow.contrib.keras
#モデル作成用
from keras.models import Model
#レイヤー各種
from keras.layers import Dense, GlobalAveragePooling2D,Input
#既存学習済みモデル
from keras.applications.inception_v3 import InceptionV3
#オプティマイザ
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
#画像サイズ
img_size = 224
#チャンネル数
ch = 3
#クラス（カテゴリ）数
categories_num = 2
#学習率
lr = 1e-5
#バッチサイズ
batch_size = 16
#エポック数


In [3]:
#既存学習済みモデル読み込み
base_model = InceptionV3(include_top = False,weights = "imagenet",
                         input_tensor=Input(shape=(img_size,img_size,ch)))
#レイヤー概要表示
#base_model.summary()

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
#特徴量抽出部分を流用する
x = base_model.output
x = GlobalAveragePooling2D()(x)
#新たな全結合層
x = Dense(1024,activation = 'relu')(x)
prediction = Dense(categories_num,activation = 'softmax')(x)
model = Model(inputs = base_model.input,output = prediction)
#レイヤー概要表示（全結合層が変更されていることを確認）
#model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  import sys


In [5]:
#レイヤー凍結（重みを更新しないようにする）処理
#レイヤー数確認
layer_len = len(base_model.layers)
#初期状態確認
#print(layer_len)
#for layer in base_model.layers:
#    print(layer.trainable)
#とりあえず最後以外全部凍結
for layer in base_model.layers:
    layer.trainable=False
#最後だけTrue
base_model.layers[layer_len -1 ].trainable = True
#変更ができているか確認
#print(base_model.layers[layer_len -1 ].trainable)
#for layer in base_model.layers:
#    print(layer.trainable)

In [6]:
#モデルコンパイル
model.compile(optimizer=Adam(lr=lr),loss = 'categorical_crossentropy',
              metrics=['accuracy'])
#確認
#model.summary()